In [1]:
# Time - модуль для работы со временем в Python.
# Selenium WebDriver – программная библиотека для управления браузерами.
# Класс Keys обеспечивает взаимодействие с командами клавиатуры, такими как RETURN, F1, ALT и т.д.
# Action Сhains - способ автоматизации низкоуровневых взаимодействий, таких как движения мыши, взаимодействия с контекстным меню и т.д.
# SQLite – компактная встраиваемая СУБД. 
# Pandas - библиотека обработки и анализа данных.
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import sqlite3
import pandas as pd
import re


# Создаем объект connection, а также новый файл tales.bd в рабочей директории. 
# Создаем курсор - специальный объект, который делает запросы к базе данных и получает их результаты.
# Делаем запрос к базе данных. 
# IF NOT EXISTS поможет при попытке повторного подключения к базе данных. Запрос проверит, существует ли таблица. 
# Если да — проверит, ничего ли не поменялось. Создаем четыре колонки: Name, Author, Language, Text.
con = sqlite3.connect("tales.bd")
cur = con.cursor()
cur.execute("CREATE TABLE IF NOT EXISTS tales (Name TEXT, Author TEXT, Language TEXT, Text TEXT)")

# Создаем элемент класса Firefox WebDriver. Для этого должен быть установлен браузер Firefox.
# Для браузера Firefox загружаем драйвер geckodriver для своей версии ОС и прописываем путь к нему в переменной окружения PATH.
# В параметре executable_path указываем полный путь к geckodriver.exe на нашем ПК.
driver = webdriver.Firefox(executable_path = r"C:\Users\User\Desktop\All\geckodriver.exe")

# Метод driver.get перенаправляет к странице URL в параметре.
# Находим элемент класса dataTables_length (чтобы вывести весь список текстов).
# Установим значение для неявного (косвенного, скрытого) ожидания - 10 сек.
# Находим 3-й элемент с тегом "a" (его индекс = 2 (индексация начинается с 0)) - это кнопка "Все".
# Происходит имитация нажатия мышки на элемент.
driver.get("https://skazki-basni.ru/skazki/")
all_button = driver.find_element_by_class_name("dataTables_length")
driver.implicitly_wait(10)
third_button = all_button.find_elements_by_tag_name("a")[2]
ActionChains(driver).move_to_element(third_button).click(third_button).perform()

# Страница прокручивается до конца.
# Находим таблицу по id ("magic"), тело таблицы по тегу ("tbody"), строки по тегу ("tr").
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
links = driver.find_element_by_id("magic").find_element_by_tag_name("tbody").find_elements_by_tag_name("tr")

# Создаем пустой список для ссылок.
# Находим в строках (links) все ссылки с классом ("name") и атрибутом ("href"), добавляем в список.
link_strings = []
for i in links:
    link_strings.append(i.find_element_by_class_name("name").get_attribute("href"))

# В списке находим ссылки, выводим их на печать.
# Задаем значение для приостановки выполнения программы - 2 сек.
# Находим название по тегу h1.
# Находим автора (народная) (класс - "breadcrumb", тег - "span" (строка), 4-й элемент, тег "a").
# Находим страну происхождения (если кол-во элементов с тегом span = 6 (у всех народных сказок), 
# находим 5-й элемент, тег "a").
# Находим текст сказки (класс - "content", тег - "p").
# Если текст состоит из глав, находим элемент с классом "content", тег "li".
# Имитируем нажатие клавиши "ENTER" на элемент с тегом "a".
# Находим элемент с классом "content".
# Удаляем из текста нечитаемые символы.
# Разбиваем текст по символу новой строки.
# Удаляем ненужные элементы (содержание (+1) и список глав (равно кол-ву элементов lis)).
# Объединяем все части в один текст.
# Удаляем ненужные слова.
# Заменяем кавычки на одинаковые.
# Ставим пробелы после знаков препинания, которые слились с началом следующего предложения.
# Ставим пробелы перед тире, которые слились с концом предыдущего предложения.
# Ставим пробелы перед кавычками, которые слились с концом предыдущего предложения.
# Удаляем ненужные слова, которые слились со следующим словом и не удалились.
# Удаляем лишние пробелы перед запятыми.
# Вставляем данные в таблицу и сохраняем изменения.
for i in link_strings:
    driver.get(i)
    print(i)
    time.sleep(2)
    name = driver.find_element_by_tag_name("h1").text
    author = driver.find_element_by_class_name("breadcrumb").find_elements_by_tag_name("span")
    author_name = author[3].find_element_by_tag_name("a").text
    if len(author) == 6:
        language = author[4].find_element_by_tag_name("a").text
    else:
        continue        
    array = driver.find_element_by_class_name("content").find_elements_by_tag_name("p")
    tale_text = ""
    if len(array) == 0:
        lis = driver.find_element_by_class_name("content").find_elements_by_tag_name("li")
        for k in range(len(lis)):
            lis[k].find_element_by_tag_name("a").send_keys(Keys.ENTER)
            time.sleep(2)
            texts = driver.find_element_by_class_name("content").text
            texts = texts.split("\n")
            texts = texts[len(lis)+1:]
            texts = '\n'.join(texts)
            texts = texts.replace("\xad", "")
            texts = texts.replace("Следующая глава", "").replace("Предыдущая глава", "")
            texts = re.sub('(["“])(?=[А-Я])', r'«', texts)
            texts = re.sub('(?<=[а-я|,.!?])(["”])', r'»', texts)
            texts = re.sub('(["”])(?=[а-я])', r'«', texts)
            texts = re.sub('([.:»…!—?-])(?=[А-Я])', r'\1 ', texts)
            texts = re.sub('(?<=[,.!?»:|…])([—|–|-])', r' —', texts)
            texts = re.sub('(?<=[,.!?»:|…])(«)', r' «', texts)
            texts = texts.replace(" ,", ",")
            tale_text += texts
            tale_text = re.sub('(Следущая глава)(?=[А-Я])', '', tale_text)
    else:
        for y in array:
            tale_text += y.text
        tale_text = re.sub('(["“])(?=[А-Я])', r'«', tale_text)
        tale_text = re.sub('(?<=[а-я|,.!?])(["”])', r'»', tale_text)
        tale_text = re.sub('(["”])(?=[а-я])', r'«', tale_text)
        tale_text = re.sub('([.:»…!—?-])(?=[А-Я])', r'\1 ', tale_text)
        tale_text = re.sub('(?<=[,.!?»:|…])([—|–|-])', r' —', tale_text)
        tale_text = re.sub('(?<=[,.!?»:|…])(«)', r' «', tale_text)
        tale_text = tale_text.replace(" ,", ",")
        
    cur.execute("INSERT INTO tales VALUES(?,?,?,?)", [name, author_name, language, tale_text])
    con.commit()

# Сохраняем базу данных как файл excel. 
# Закрываем объект-курсор и закрываем соединение.
file_1 = pd.read_sql("SELECT * FROM tales", con)
file_1.to_excel("excel_file.xlsx", encoding="utf-8",index=False)

cur.close()
con.close()

https://skazki-basni.ru/skazki/narod/russkie/morozko/
https://skazki-basni.ru/skazki/pushkin/skazka-o-saltane/
https://skazki-basni.ru/skazki/narod/russkie/kolobok/
https://skazki-basni.ru/skazki/aleksi-tolstoy/zolotoj-kljuchik-ili-prikljuchenija-buratino/
https://skazki-basni.ru/skazki/pushkin/skazka-o-rybake-i-rybke/
https://skazki-basni.ru/skazki/narod/russkie/teremok/
https://skazki-basni.ru/skazki/narod/russkie/repka/
https://skazki-basni.ru/skazki/pushkin/skazka-o-semi-bogatyrjah/
https://skazki-basni.ru/skazki/narod/russkie/kroshechka-havroshechka/
https://skazki-basni.ru/skazki/narod/russkie/tri-medvedja/
https://skazki-basni.ru/skazki/narod/russkie/kurochka-rjaba/
https://skazki-basni.ru/skazki/narod/russkie/masha-i-medved/
https://skazki-basni.ru/skazki/narod/russkie/gusi-lebedi-2/
https://skazki-basni.ru/skazki/perro/krasnaja-shapochka/
https://skazki-basni.ru/skazki/narod/russkie/po-shhuchemu-veleniju/
https://skazki-basni.ru/skazki/perro/zolushka/
https://skazki-basni.ru/s

https://skazki-basni.ru/skazki/narod/russkie/elena-premudraja/
https://skazki-basni.ru/skazki/narod/russkie/doch-semiletka/
https://skazki-basni.ru/skazki/narod/ketskie/lisa/
https://skazki-basni.ru/skazki/narod/slovenskie/skazka-lis/
https://skazki-basni.ru/skazki/narod/russkie/belaja-utochka/
https://skazki-basni.ru/skazki/narod/russkie/lisa-i-rak/
https://skazki-basni.ru/skazki/grimm/korolek/
https://skazki-basni.ru/skazki/narod/russkie/soldatskaja-shinel/
https://skazki-basni.ru/skazki/lev-tolstoy/kak-volki-uchat-svoih-detej-rasskaz/
https://skazki-basni.ru/skazki/mamin-sibiryak/skazka-pro-carja-goroha/
https://skazki-basni.ru/skazki/narod/russkie/sem-simeonov/
https://skazki-basni.ru/skazki/narod/russkie/kak-sobaka-druga-iskala/
https://skazki-basni.ru/skazki/narod/francuzskie/skazka-pro-sinjuju-borodu/
https://skazki-basni.ru/skazki/grimm/gans-moj-ezh/
https://skazki-basni.ru/skazki/narod/russkie/ivan-krestjanskij-syn-i-chudo-judo/
https://skazki-basni.ru/skazki/narod/russkie/ter

https://skazki-basni.ru/skazki/narod/russkie/carevna-zmeja/
https://skazki-basni.ru/skazki/narod/latyshskie/durak/
https://skazki-basni.ru/skazki/narod/indijskie/zolotaja-ryba/
https://skazki-basni.ru/skazki/prishvin/vremena-goda/
https://skazki-basni.ru/skazki/narod/russkie/dobroe-slovo/
https://skazki-basni.ru/skazki/narod/bolgarskie/kap-kap/
https://skazki-basni.ru/skazki/narod/avarskie/lisa-i-gus/
https://skazki-basni.ru/skazki/narod/russkie/muzhik-i-barin/
https://skazki-basni.ru/skazki/bazhov/gornyj-master/
https://skazki-basni.ru/skazki/harris/kak-bratec-krolik-perehitril-bratca-lisa/
https://skazki-basni.ru/skazki/narod/ukrainskie/skazka-skazka-pro-ivasika/
https://skazki-basni.ru/skazki/lev-tolstoy/kak-muzhik-ubral-kamen-byl/
https://skazki-basni.ru/skazki/narod/karachaevskie/medved-volk-i-lisa/
https://skazki-basni.ru/skazki/narod/belorusskie/aljonka/
https://skazki-basni.ru/skazki/lev-tolstoy/kak-vor-sam-sebja-vydal-byl/
https://skazki-basni.ru/skazki/narod/russkie/kak-lisa-

https://skazki-basni.ru/skazki/narod/russkie/lisa-i-medved/
https://skazki-basni.ru/skazki/narod/jakutskie/skazka-kak-veter-k-velikoj-gore-hodil/
https://skazki-basni.ru/skazki/narod/tadzhikskie/skazka-glupyj-volk/
https://skazki-basni.ru/skazki/narod/francuzskie/skazka-korol-voron/
https://skazki-basni.ru/skazki/narod/birmanskie/siroty/
https://skazki-basni.ru/skazki/lev-tolstoy/dlja-chego-veter-rassuzhdenie/
https://skazki-basni.ru/skazki/harris/kak-bratec-krolik-lishilsja-hvosta/
https://skazki-basni.ru/skazki/narod/francuzskie/skazka-belyj-olen/
https://skazki-basni.ru/skazki/narod/kambodzhijskie/vezhlivyj-krolik/
https://skazki-basni.ru/skazki/narod/udjegejskie/skazka-brat-i-sestra/
https://skazki-basni.ru/skazki/narod/altajskie/sartakpaj/
https://skazki-basni.ru/skazki/harris/hrabryj-bratec-opossum/
https://skazki-basni.ru/skazki/narod/korejskie/chetyre-zhelanija/
https://skazki-basni.ru/skazki/garin-mihailovsky/schaste-garin-mihajlovskij/
https://skazki-basni.ru/skazki/narod/jap

https://skazki-basni.ru/skazki/narod/brazilskie/koldunja/
https://skazki-basni.ru/skazki/narod/bosnijskie/barsuk-i-lisa/
https://skazki-basni.ru/skazki/narod/afganskie/vernoe-sredstvo/
https://skazki-basni.ru/skazki/narod/anglijskie/binnori/
https://skazki-basni.ru/skazki/narod/altajskie/strashnyj-gost/
https://skazki-basni.ru/skazki/zoschenko/velikie-puteshestvenniki/
https://skazki-basni.ru/skazki/prishvin/berestjanaja-trubochka/
https://skazki-basni.ru/skazki/narod/tadzhikskie/skazka-farhad-i-shirin/
https://skazki-basni.ru/skazki/narod/latyshskie/volk-i-muzhik/
https://skazki-basni.ru/skazki/narod/karelskie/belka-rukavica-i-igolka/
https://skazki-basni.ru/skazki/narod/afganskie/net-huda-bez-dobra/
https://skazki-basni.ru/skazki/narod/anglijskie/vladyka-iz-vladyk/
https://skazki-basni.ru/skazki/narod/armjanskie/portnoj-i-car/
https://skazki-basni.ru/skazki/narod/albanskie/petuh-i-kurica/
https://skazki-basni.ru/skazki/narod/abhazskie/mudrost/
https://skazki-basni.ru/skazki/gauf/skaz

...

https://skazki-basni.ru/skazki/narod/avarskie/kak-lisa-uvidela-lunu/
https://skazki-basni.ru/skazki/narod/avarskie/bogachu-ne-podobaet-byt-melochnym/
https://skazki-basni.ru/skazki/narod/abhazskie/shardyn-i-kapluny/
https://skazki-basni.ru/skazki/narod/abhazskie/tri-brata-i-knjaz/
https://skazki-basni.ru/skazki/narod/abhazskie/radost-petuha/
https://skazki-basni.ru/skazki/narod/abhazskie/pokladistaja-ljagushka/
https://skazki-basni.ru/skazki/narod/abhazskie/obidchivaja-staruha/
https://skazki-basni.ru/skazki/narod/abhazskie/mach-i-knjaz/
https://skazki-basni.ru/skazki/narod/abhazskie/kurinyj-zheludok/
https://skazki-basni.ru/skazki/lev-tolstoy/glupyj-muzhik-bestolkovyj-muzhik-skazka/
https://skazki-basni.ru/skazki/garin-mihailovsky/volshebnica-asham-garin-mihajlovskij/
https://skazki-basni.ru/skazki/bazhov/sochnevye-kameshki/
https://skazki-basni.ru/skazki/bazhov/krugovoj-fonar/
https://skazki-basni.ru/skazki/bazhov/dorogoj-zemli-vitok/
https://skazki-basni.ru/skazki/prishvin/golubaja-

https://skazki-basni.ru/skazki/narod/irlandskie/tjomnyj-patrik-i-povelitel-voron-kromahi/
https://skazki-basni.ru/skazki/narod/irlandskie/dik-i-ego-zhena-morskaja-deva/
https://skazki-basni.ru/skazki/narod/iranskie/zolotovolosyj-totambaj/
https://skazki-basni.ru/skazki/narod/indonezijskie/pochemu-u-slepnja-dva-glaza/
https://skazki-basni.ru/skazki/narod/indonezijskie/kak-la-dana-sel-zhertvu-bogam/
https://skazki-basni.ru/skazki/narod/indijskie/tri-figurki/
https://skazki-basni.ru/skazki/narod/indijskie/plotnik-i-ptichka-karvaki/
https://skazki-basni.ru/skazki/narod/indijskie/pataliputra/
https://skazki-basni.ru/skazki/narod/indijskie/koren-dobra-ne-sohnet/
https://skazki-basni.ru/skazki/narod/ingushskie/chja-vina/
https://skazki-basni.ru/skazki/narod/datskie/hans-silach/
https://skazki-basni.ru/skazki/narod/daosskie/spor-o-solnce/
https://skazki-basni.ru/skazki/narod/daosskie/pjat-lekarstv/
https://skazki-basni.ru/skazki/narod/gruzinskie/syn-tushina-i-carevich/
https://skazki-basni.ru/

https://skazki-basni.ru/skazki/narod/osetinskie/skazka-vdovij-syn/
https://skazki-basni.ru/skazki/narod/norvezhskie/vecher-provedjonnyj-v-odnoj-norvezhskoj-kuhne/
https://skazki-basni.ru/skazki/narod/nivhskie/medved-i-bratja/
https://skazki-basni.ru/skazki/narod/nepalskie/sud-panchej/
https://skazki-basni.ru/skazki/narod/nepalskie/vorobej-i-goroshina/
https://skazki-basni.ru/skazki/narod/nanajskie/sjenuje-bogatyr/
https://skazki-basni.ru/skazki/narod/moldavskie/turok-i-devushka/
https://skazki-basni.ru/skazki/narod/marijskie/storozh-pchjol/
https://skazki-basni.ru/skazki/narod/makedonskie/boltlivaja-zhena-i-umnaja-staruha/
https://skazki-basni.ru/skazki/narod/madagaskarskie/troe-bratev-s-hvostami/
https://skazki-basni.ru/skazki/narod/latyshskie/otcovy-sovety/
https://skazki-basni.ru/skazki/narod/latyshskie/mudraja-barynja/
https://skazki-basni.ru/skazki/narod/kumykskie/pejma-hanum/
https://skazki-basni.ru/skazki/narod/kubinskie/izbavilsja/
https://skazki-basni.ru/skazki/narod/korejskie

https://skazki-basni.ru/skazki/narod/shotlandskie/skazka-ujsden-mor-i-zhjoltaja-bezrogaja-koza/
https://skazki-basni.ru/skazki/narod/shorskie/skazka-dogadlivaja-vydra/
https://skazki-basni.ru/skazki/narod/shvejcarskie/skazka-ploshhadka-dlja-tancev-v-granvillare/
https://skazki-basni.ru/skazki/narod/chuvashskie/skazka-bolshoj-zajac/
https://skazki-basni.ru/skazki/narod/cheshskie/skazka-serebrjanye-sosulki-v-radgoshhi/
https://skazki-basni.ru/skazki/narod/cheshskie/skazka-kum-matej-i-kum-irzhi/
https://skazki-basni.ru/skazki/narod/cyganskie/skazka-kak-cyganka-cherta-otvadila/
https://skazki-basni.ru/skazki/narod/cyganskie/skazka-kak-cygan-vorovat-otuchilsja/
https://skazki-basni.ru/skazki/narod/francuzskie/skazka-jeujen-kongar/
https://skazki-basni.ru/skazki/narod/francuzskie/skazka-hitryj-kjure/
https://skazki-basni.ru/skazki/narod/francuzskie/skazka-staryj-gorshok-s-zolotymi-jekju/
https://skazki-basni.ru/skazki/narod/francuzskie/skazka-princessa-markassa-i-ptica-dredejn/
https://skazk

https://skazki-basni.ru/skazki/narod/birmanskie/kak-za-dobro-vozdaetsja-dobrom/
https://skazki-basni.ru/skazki/narod/birmanskie/zheny-korolja-natov/
https://skazki-basni.ru/skazki/narod/birmanskie/bogach-i-lentjaj/
https://skazki-basni.ru/skazki/narod/afrikanskie/o-kuropatke-nangumbi-i-leoparde-ulu/
https://skazki-basni.ru/skazki/narod/afrikanskie/nijkang-i-dimo/
https://skazki-basni.ru/skazki/narod/afrikanskie/kolodec-njamy/
https://skazki-basni.ru/skazki/narod/afganskie/umnyj-karlik/
https://skazki-basni.ru/skazki/narod/afganskie/vor-i-mudrec/
https://skazki-basni.ru/skazki/narod/assirijskie/bolotnyj-kulik-i-ohotnik/
https://skazki-basni.ru/skazki/narod/anglijskie/chernaja-rubashka/
https://skazki-basni.ru/skazki/narod/anglijskie/vygodnaja-sdelka/
https://skazki-basni.ru/skazki/narod/angolskie/skazka-pro-sha-shipo-provalivshegosja-v-peshheru/
https://skazki-basni.ru/skazki/narod/angolskie/antilopa-sesha-i-lev/
https://skazki-basni.ru/skazki/narod/amerikanskie/neposeda-i-da-skorej/
ht